# Изучение сервиса для чтения книг

Цель исследования: Проанализировать базу данных.  Выявить интересные закономерности и факты.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [14]:
a = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(a), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
b = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(b), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [12]:
c = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(c), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
d = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(d), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
e = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(e), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Сколько книг вышло после 1 января 2000 года

После 1 января 2000 года было выпущено 819 книг

In [7]:
count='''SELECT COUNT(book_id) FROM books WHERE publication_date > '01.01.2000' '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(count), con = con)

,count
0,819


### Количество обзоров и средняя оценка для каждой книги

In [21]:
rating='''WITH 
re AS (SELECT book_id, COUNT(text) FROM reviews GROUP BY book_id ORDER BY book_id),
rr AS (SELECT book_id, AVG (rating) FROM ratings GROUP BY book_id ORDER BY book_id)
SELECT b.title,b.book_id, re.count, rr.avg
FROM books AS b
LEFT JOIN rr ON b.book_id=rr.book_id
LEFT JOIN re ON b.book_id=re.book_id
'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(rating), con = con)

,title,book_id,count,avg
0,'Salem's Lot,1,2.0,3.666667
1,1 000 Places to See Before You Die,2,1.0,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3.0,4.666667
3,1491: New Revelations of the Americas Before C...,4,2.0,4.500000
4,1776,5,4.0,4.000000
...,...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,672,NaN,5.000000
996,Anne Rice's The Vampire Lestat: A Graphic Novel,83,NaN,3.666667
997,Essential Tales and Poems,221,NaN,4.000000
998,Leonardo's Notebooks,387,NaN,4.000000


### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры


Проведя расчеты мы выяснили что больше всего книг толще 50 страниц выпостило издательство Penguin Books с 42 книгами. Так же в тройку лидеров вошли Vintage	 - 31 книга и Grand Central Publishing	- 25 книг.

In [9]:
publish='''WITH cou AS(SELECT publisher_id, COUNT(book_id) AS count_books FROM books WHERE num_pages > 50 GROUP BY publisher_id ORDER BY COUNT(book_id) DESC)
         SELECT pu.publisher, cou.count_books FROM publishers AS pu INNER JOIN cou on pu.publisher_id=cou.publisher_id '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(publish), con = con)

,publisher,count_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Ballantine Books (NY),1
330,Plaza y Janés,1
331,HarperCollinsPublishers,1
332,Random House: Modern Library,1


#### Определим автора с самой высокой средней оценкой книг — учитываем только книги с 50 и более оценками


Мы выяснили что автором книг с самой высокой средней оченкой, среди книг имеющих более 50 оценок, оказался J.K. Rowling/Mary GrandPré со средней оценкой 4,283.В тройкку так же вошли Markus Zusak/Cao Xuân Việt Khương с оценкой 4,264 и J.R.R. Tolkien с оценкой 4,258.

In [10]:
book='''WITH bok AS (SELECT book_id, COUNT (rating) AS count_rating,AVG(rating) AS avg_rating FROM ratings GROUP BY book_id HAVING COUNT (rating) > 50 ORDER BY avg_rating DESC)
        SELECT au.author, AVG(bok.avg_rating) FROM authors AS au INNER JOIN books AS bo ON bo.author_id=au.author_id
        INNER JOIN bok ON bok.book_id=bo.book_id GROUP BY au.author ORDER BY avg DESC  '''
con=engine.connect()

pd.io.sql.read_sql(sql=text(book), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

Среднее количество обзоров от пользователей, поставивших более 48 оценок, составило 24 обзора.

In [11]:
review='''WITH u AS (SELECT re.username, COUNT(re.text) FROM reviews AS re INNER JOIN (SELECT username, COUNT(rating) FROM ratings GROUP BY username HAVING COUNT(rating) > 48) AS d ON re.username=d.username
GROUP BY re.username)
SELECT AVG(count) FROM u'''
con=engine.connect()

pd.io.sql.read_sql(sql=text(review), con = con)

,avg
0,24.0


### Вывод:

В ходе исследовани я нам удалось выяснить следующее:

После 1 января 2000 года было выпущено 819 книг

среди книг с самой высокой средней оценкой такие книги как A Woman of Substance (Emma Harte Saga #1),Dead Souls,Crucial Conversations: Tools for Talking When, Tai-Pan (Asian Saga #2),Wherever You Go There You Are: Mindfulness Me

Больше всего книг толщиной более 50 страниц выпустило издательство Penguin Books-42. Так же в трой ку лидеров вошли Vintage-31 и Grand Central Publishing-25

Лидером по самой высокой средней оценке среди авторов оказалась J.K. Rowling/Mary GrandPré со средней оценко 4.283844. В тройку так же попали Markus Zusak/Cao Xuân Việt Khương-4.264151 и J.R.R. Tolkien-4.258446.

Так же выяснили что среднее количество отзывов среди пользователей,ставящих не менее 48 оценок -24 отзыва


